In [2]:
import numpy as np
import pandas as pd
import os
import requests
import io
from datetime import datetime, timedelta
import multiprocessing
import arviz as az
import logging
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
import xgboost as xgb
from scipy.stats import poisson
import sqlite3

In [17]:
conn = sqlite3.connect(r"C:\Users\Owner\dev\team-eval\match_db.db")
df = pd.read_sql_query("""SELECT * FROM btb_matches""", conn)

df

,division_id,division,season_id,season,match_date,match_teams,home_team_id,home_team,away_team_id,away_team,...,home_deep,away_deep,home_ppda,away_ppda,bet365_home_odds,bet365_draw_odds,bet365_away_odds,bet365_u25_odds,bet365_o25_odds,match_id
0,98b8784f6685b7289f583e0ce4b4f6f2,Premier League,3ac445d3cc1d404987efdfcfa42f3bcd,20242025,2024-08-16 00:00:00,Man United - Fulham,f2b82cdbdadf9d3ec47c3a6be66dcfad,Man United,8cd5e94668b139c1f42a89a1e130f3cf,Fulham,...,7.0,3.0,7.379310,10.833333,1.60,4.20,5.25,2.50,1.53,Man United - Fulham_20240816
1,98b8784f6685b7289f583e0ce4b4f6f2,Premier League,3ac445d3cc1d404987efdfcfa42f3bcd,20242025,2024-08-17 00:00:00,Newcastle - Southampton,78e9266876e7649e0a12e3840f5be006,Newcastle,5a884401673693b0bdf379fefb7ec2b2,Southampton,...,4.0,13.0,16.250000,3.789474,1.36,5.25,8.00,3.00,1.40,Newcastle - Southampton_20240817
2,98b8784f6685b7289f583e0ce4b4f6f2,Premier League,3ac445d3cc1d404987efdfcfa42f3bcd,20242025,2024-08-17 00:00:00,Arsenal - Wolves,4fc9baf210346939946d5a49f255588b,Arsenal,31f3ecbc5c48590ccc7dabaedd49a4ff,Wolves,...,14.0,2.0,7.769231,10.818182,1.18,7.50,13.00,2.75,1.44,Arsenal - Wolves_20240817
3,98b8784f6685b7289f583e0ce4b4f6f2,Premier League,3ac445d3cc1d404987efdfcfa42f3bcd,20242025,2024-08-17 00:00:00,Nott'm Forest - Bournemouth,9a8e1e9fad8766fc3d69a0c26d98b928,Nott'm Forest,b436d55f36cfbe8a085c8b75fb7fe98a,Bournemouth,...,10.0,4.0,8.653846,9.954545,2.45,3.50,2.80,2.10,1.73,Nott'm Forest - Bournemouth_20240817
4,98b8784f6685b7289f583e0ce4b4f6f2,Premier League,3ac445d3cc1d404987efdfcfa42f3bcd,20242025,2024-08-17 00:00:00,Ipswich - Liverpool,e4f63bf6d6d2cd121e6c8e59bef68209,Ipswich,afce84ff226407a47c9782a742ba02f7,Liverpool,...,2.0,13.0,18.777778,8.739130,8.50,5.50,1.33,3.00,1.40,Ipswich - Liverpool_20240817
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4468,c4781225ef2d5018f7a9df4b6cb1c1fe,Championship,a290f895d08db6bb30a573aedf784bad,20202021,2021-05-08 00:00:00,Watford - Swansea,6e424e2cb43e11ef176c2a8dcb72f9de,Watford,013d5e8c67e9155030c74930aaf5be33,Swansea,...,NaN,NaN,NaN,NaN,2.00,3.40,3.80,1.85,2.00,Watford - Swansea_20210508
4469,c4781225ef2d5018f7a9df4b6cb1c1fe,Championship,a290f895d08db6bb30a573aedf784bad,20202021,2021-05-08 00:00:00,Middlesbrough - Wycombe,b4cb58c3a936665786359c2c634d6ff4,Middlesbrough,c7e249acd30a69cea63bc51a66e08807,Wycombe,...,NaN,NaN,NaN,NaN,1.75,4.00,4.33,2.10,1.72,Middlesbrough - Wycombe_20210508
4470,c4781225ef2d5018f7a9df4b6cb1c1fe,Championship,a290f895d08db6bb30a573aedf784bad,20202021,2021-05-08 00:00:00,Coventry - Millwall,9ed47047fb3b7f26ada9a35fe83d7b1b,Coventry,ea11a8d754f9bbf49dc01d6d3ae875d9,Millwall,...,NaN,NaN,NaN,NaN,2.37,3.50,2.87,1.80,2.00,Coventry - Millwall_20210508
4471,c4781225ef2d5018f7a9df4b6cb1c1fe,Championship,a290f895d08db6bb30a573aedf784bad,20202021,2021-05-08 00:00:00,Reading - Huddersfield,26fccddf2f94b1685b184267996e10f8,Reading,5668c64f7d3e02ee708188dbe0fdaa4e,Huddersfield,...,NaN,NaN,NaN,NaN,1.95,3.75,3.50,2.00,1.80,Reading - Huddersfield_20210508


In [18]:
final_df = pd.read_sql_query("""SELECT * FROM xgb_match_features_0065_001 WHERE division = 'Premier League'""", conn)

final_df[final_df["match_date"] > '2024-08-01'].to_csv("test_data.csv")

In [76]:
final_df["match_date"] = pd.to_datetime(final_df["match_date"])

final_df = final_df[final_df["match_date"] > '2020-12-31']

#final_df = final_df[final_df["division"] == "Premier League"]
final_df["xg_interaction"] = final_df["rolling_xg_for"] * final_df["opponent_rolling_xg_against"]
final_df["rolling_finishing_ability"] = final_df["rolling_goals_for"] - final_df["rolling_xg_for"]
final_df["rolling_opponent_finishing_ability"] = final_df["opponent_rolling_goals_against"] - final_df["opponent_rolling_xg_against"]

feature_cols = ["division", "home?", "avg_market_value", "opponent_avg_market_value", "rolling_finishing_ability", "rolling_opponent_finishing_ability", #"team_season", "opponent_team_season",#"xg_interaction",
             "rolling_goals_for", "rolling_xg_for", "rolling_shots_for", "rolling_shots_on_target_for", "rolling_corners_for", "rolling_deep_for", "rolling_ppda_for", 'rolling_goals_for_30d', 'rolling_goals_against_30d', 'rolling_xg_for_30d', 'rolling_xg_against_30d', 'rolling_shots_for_30d', 'rolling_shots_against_30d', 'rolling_shots_on_target_for_30d', 'rolling_shots_on_target_against_30d', 'rolling_corners_for_30d', 'rolling_corners_against_30d', 'rolling_deep_for_30d', 'rolling_deep_against_30d', 'rolling_ppda_for_30d', 'rolling_ppda_against_30d', 
             "opponent_rolling_goals_against", "opponent_rolling_xg_against", "opponent_rolling_shots_against", "opponent_rolling_shots_on_target_against", "opponent_rolling_corners_against", "opponent_rolling_deep_against", "opponent_rolling_ppda_against", 'opponent_rolling_goals_for_30d', 'opponent_rolling_goals_against_30d', 'opponent_rolling_xg_for_30d', 'opponent_rolling_xg_against_30d', 'opponent_rolling_shots_for_30d', 'opponent_rolling_shots_against_30d', 'opponent_rolling_shots_on_target_for_30d', 'opponent_rolling_shots_on_target_against_30d', 'opponent_rolling_corners_for_30d', 'opponent_rolling_corners_against_30d', 'opponent_rolling_deep_for_30d', 'opponent_rolling_deep_against_30d', 'opponent_rolling_ppda_for_30d', 'opponent_rolling_ppda_against_30d']

X = final_df[feature_cols]

# Convert categorical columns to category type
cat_cols = ["division"] #"team", "opponent_team"
for col in cat_cols:
    X[col] = X[col].astype('category')


y = final_df["goals"] 

text_cutoff = pd.to_datetime('2024-08-01')
X_train = X[final_df['match_date'] < text_cutoff]
y_train = y[final_df['match_date'] < text_cutoff]
X_test = X[final_df['match_date'] >= text_cutoff]
y_test = y[final_df['match_date'] >= text_cutoff]

print(f"Training set size: {X_train.shape[0]} samples")
print(f"Test set size: {X_test.shape[0]} samples")

model_params = {
        'n_estimators': 100,
        'learning_rate': 0.03,
        'max_depth': 5,
        'subsample': 0.7,
        'colsample_bytree': 0.9,
        'min_child_weight': 3,
        'gamma': 0.1,
        'alpha': 1,
        #'lambda_': 1,
        'random_state': 26,
        #'enable_categorical': True
    }


# Train an XGBoost model
model = xgb.XGBRegressor(objective='count:poisson', **model_params, enable_categorical= True)

model.fit(X_train, y_train)

# Make predictions
y_train_pred = model.predict(X_train)
y_pred = model.predict(X_test)

# Evaluate model
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
train_mae = mean_absolute_error(y_train, y_train_pred)
print(f"RMSE: {rmse:.4f}")
print(f"Test MAE: {mae:.4f}")
print(f"Train MAE: {train_mae:.4f}")

X_test_has_nan = X_test.isna().any(axis=1)
y_test_has_nan = y_test.isna()

print(f"X_test rows with NaN: {X_test_has_nan.sum()}")
print(f"y_test rows with NaN: {y_test_has_nan.sum()}")

C:\Users\Owner\AppData\Local\Temp\ipykernel_4880\2447514632.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = X[col].astype('category')


Training set size: 2730 samples
Test set size: 578 samples
RMSE: 1.1781
Test MAE: 0.9275
Train MAE: 0.8415
X_test rows with NaN: 2
y_test rows with NaN: 0


In [77]:
# Get the test set indexes
test_indices = final_df.index[final_df['match_date'] >= text_cutoff]

# Create a temporary dataframe with all the test data
temp_df = final_df.loc[test_indices].copy()

# Initialize the predictions dataframe
match_preds = []

# Group by match_id to combine home and away rows
for match_id, group in temp_df.groupby('match_id'):
    if len(group) != 2:
        print(f"Warning: Match ID {match_id} does not have exactly 2 rows (has {len(group)})")
        continue
        
    # Get home and away rows
    home_row = group[group['home?'] == 1]
    away_row = group[group['home?'] == 0]
    
    if len(home_row) != 1 or len(away_row) != 1:
        print(f"Warning: Match ID {match_id} doesn't have one home and one away team")
        continue
    
    # Get the model predictions for both teams
    home_idx = home_row.index[0]
    away_idx = away_row.index[0]
    
    home_pred = y_pred[np.where(test_indices == home_idx)[0][0]] if home_idx in test_indices else None
    away_pred = y_pred[np.where(test_indices == away_idx)[0][0]] if away_idx in test_indices else None
    
    # Create a single row for this match
    match_data = {
        'match_id': match_id,
        'match_date': home_row['match_date'].values[0],
        'home_team': home_row['team'].values[0],
        'away_team': away_row['team'].values[0],
        'division': home_row['division'].values[0],
        'actual_home_goals': home_row['goals'].values[0],
        'actual_away_goals': away_row['goals'].values[0],
        'predicted_home_goals': home_pred,
        'predicted_away_goals': away_pred,
        'home_error': home_row['goals'].values[0] - home_pred if home_pred is not None else None,
        'away_error': away_row['goals'].values[0] - away_pred if away_pred is not None else None
    }
    
    # Add match_teams if available
    if 'match_teams' in home_row.columns:
        match_data['match_teams'] = home_row['match_teams'].values[0]
    
    match_preds.append(match_data)

# Create the final predictions dataframe
preds_df = pd.DataFrame(match_preds)

# Add some calculated columns
preds_df['home_abs_error'] = np.abs(preds_df['home_error'])
preds_df['away_abs_error'] = np.abs(preds_df['away_error'])
preds_df['total_abs_error'] = preds_df['home_abs_error'] + preds_df['away_abs_error']

# Save to CSV
#preds_df.to_csv("match_predictions.csv", index=False)

# Display summary stats
print("\nPrediction Summary:")
print(f"Total matches predicted: {len(preds_df)}")
print(f"Home goals MAE: {preds_df['home_abs_error'].mean():.4f}")
print(f"Away goals MAE: {preds_df['away_abs_error'].mean():.4f}")
print(f"Overall MAE: {(preds_df['home_abs_error'].mean() + preds_df['away_abs_error'].mean())/2:.4f}")
print(f"Date range: {preds_df['match_date'].min().date()} to {preds_df['match_date'].max().date()}")

preds_df


Prediction Summary:
Total matches predicted: 289
Home goals MAE: 0.9636
Away goals MAE: 0.8915
Overall MAE: 0.9275
Date range: 2024-08-16 to 2025-03-16


,match_id,match_date,home_team,away_team,division,actual_home_goals,actual_away_goals,predicted_home_goals,predicted_away_goals,home_error,away_error,home_abs_error,away_abs_error,total_abs_error
0,Arsenal - Aston Villa_20250118,2025-01-18,Arsenal,Aston Villa,Premier League,2,2,1.654242,1.030796,0.345758,0.969204,0.345758,0.969204,1.314963
1,Arsenal - Brighton_20240831,2024-08-31,Arsenal,Brighton,Premier League,1,1,1.896443,1.107668,-0.896443,-0.107668,0.896443,0.107668,1.004112
2,Arsenal - Chelsea_20250316,2025-03-16,Arsenal,Chelsea,Premier League,1,0,1.951345,1.099479,-0.951345,-1.099479,0.951345,1.099479,2.050824
3,Arsenal - Everton_20241214,2024-12-14,Arsenal,Everton,Premier League,0,0,2.165478,0.880981,-2.165478,-0.880981,2.165478,0.880981,3.046458
4,Arsenal - Ipswich_20241227,2024-12-27,Arsenal,Ipswich,Premier League,1,0,2.494205,0.801385,-1.494205,-0.801385,1.494205,0.801385,2.295591
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,Wolves - Man City_20241020,2024-10-20,Wolves,Man City,Premier League,1,2,1.053363,2.216520,-0.053363,-0.216520,0.053363,0.216520,0.269883
285,Wolves - Man United_20241226,2024-12-26,Wolves,Man United,Premier League,2,0,1.283266,1.484760,0.716734,-1.484760,0.716734,1.484760,2.201494
286,Wolves - Newcastle_20240915,2024-09-15,Wolves,Newcastle,Premier League,1,2,1.313203,1.881419,-0.313203,0.118581,0.313203,0.118581,0.431784
287,Wolves - Nott'm Forest_20250106,2025-01-06,Wolves,Nott'm Forest,Premier League,0,3,1.232422,1.250860,-1.232422,1.749140,1.232422,1.749140,2.981562


In [78]:
def append_df_to_sqlite_table(df_new, db_path, table_name, key_columns=None, batch_size=500):
    # Make a copy to avoid modifying the original dataframe
    df = df_new.copy()
    
    # Connect to the database
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # Check if table exists
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name=?", (table_name,))
    table_exists = cursor.fetchone() is not None
    
    if table_exists:
        # If key_columns not specified, use all columns
        if key_columns is None:
            key_columns = list(df.columns)
        
        # Get columns from existing table
        cursor.execute(f"PRAGMA table_info({table_name})")
        existing_columns = [row[1] for row in cursor.fetchall()]
        
        # Ensure all key columns exist in both dataframe and table
        key_columns = [col for col in key_columns if col in df.columns and col in existing_columns]
        
        if key_columns:
            # Process in batches to avoid SQL expression tree too large error
            filtered_dfs = []
            
            # Split dataframe into batches
            total_rows = len(df)
            for start_idx in range(0, total_rows, batch_size):
                end_idx = min(start_idx + batch_size, total_rows)
                batch_df = df.iloc[start_idx:end_idx]
                
                # For this batch, extract all key column values
                key_values = {}
                for key in key_columns:
                    key_values[key] = batch_df[key].tolist()
                
                # Build an optimized query - using IN clause for each key column
                where_clauses = []
                params = []
                
                for key in key_columns:
                    if len(key_values[key]) > 0:
                        placeholders = ','.join(['?'] * len(key_values[key]))
                        where_clauses.append(f"{key} IN ({placeholders})")
                        params.extend(key_values[key])
                
                if where_clauses:
                    # Get existing records
                    query = f"SELECT {', '.join(key_columns)} FROM {table_name} WHERE {' OR '.join(where_clauses)}"
                    cursor.execute(query, params)
                    existing_rows = cursor.fetchall()
                    
                    # Create a set of tuples representing existing rows
                    if key_columns:
                        existing_keys = set(existing_rows)
                        
                        # Filter out rows that already exist
                        batch_filtered = batch_df[~batch_df[key_columns].apply(tuple, axis=1).isin(existing_keys)]
                        if not batch_filtered.empty:
                            filtered_dfs.append(batch_filtered)
                else:
                    # No valid key values, include all rows
                    filtered_dfs.append(batch_df)
            
            # Combine all filtered batches
            if filtered_dfs:
                df_filtered = pd.concat(filtered_dfs, ignore_index=True)
                if not df_filtered.empty:
                    df_filtered.to_sql(table_name, conn, if_exists='append', index=False)
                    print(f"Added {len(df_filtered)} new records to table {table_name}")
                else:
                    print("No new records to append.")
            else:
                print("No new records to append.")
        else:
            # If no valid key columns, append all data (may create duplicates)
            df.to_sql(table_name, conn, if_exists='append', index=False)
            print(f"Warning: No valid key columns. Added all {len(df)} records to table {table_name}")
    else:
        # If table doesn't exist, create it with the new data
        df.to_sql(table_name, conn, index=False)
        print(f"Created new table {table_name} with {len(df)} records")
    
    # Close the connection
    conn.close()

append_df_to_sqlite_table(preds_df, 'match_db.db', 'xgb_pred_0065_001_prem20242025', key_columns=['match_id'])

No new records to append.


## Backtesting my model on 1x2 and o/u 2.5

In [79]:
backtest_df = pd.read_sql_query("""
    SELECT
        preds.match_id,
        preds.predicted_home_goals,
        preds.actual_home_goals,
        preds.predicted_away_goals,
        preds.actual_away_goals,
        btb.bet365_home_odds,
        btb.bet365_draw_odds,
        btb.bet365_away_odds,
        btb.bet365_u25_odds,
        btb.bet365_o25_odds
    FROM
        xgb_pred_0065_001_prem20242025 preds
    JOIN
        btb_matches btb
            ON btb.match_id = preds.match_id
                                
    """, conn)

backtest_df

,match_id,predicted_home_goals,actual_home_goals,predicted_away_goals,actual_away_goals,bet365_home_odds,bet365_draw_odds,bet365_away_odds,bet365_u25_odds,bet365_o25_odds
0,Arsenal - Aston Villa_20250118,1.590694,2,1.074542,2,1.50,4.10,7.00,2.03,1.87
1,Arsenal - Brighton_20240831,1.843244,1,1.144221,1,1.33,5.50,8.50,2.50,1.53
2,Arsenal - Chelsea_20250316,1.742448,1,1.133815,0,1.80,3.60,4.50,1.92,1.86
3,Arsenal - Everton_20241214,2.030706,0,0.932608,0,1.22,6.50,12.00,2.20,1.67
4,Arsenal - Ipswich_20241227,2.246113,1,0.899718,0,1.14,9.00,15.00,3.00,1.40
...,...,...,...,...,...,...,...,...,...,...
284,Wolves - Man City_20241020,1.058062,1,2.087760,2,8.00,6.00,1.33,2.75,1.44
285,Wolves - Man United_20241226,1.316546,2,1.455159,0,3.90,3.70,1.91,2.20,1.67
286,Wolves - Newcastle_20240915,1.290198,1,1.855103,2,3.40,3.50,2.10,2.38,1.57
287,Wolves - Nott'm Forest_20250106,1.245404,0,1.231543,3,3.30,3.20,2.30,1.73,2.10


In [80]:
def simulate_match_outcomes(home_goals, away_goals, num_simulations=10000):

    # Generate random samples from Poisson distributions
    home_scores = np.random.poisson(home_goals, num_simulations)
    away_scores = np.random.poisson(away_goals, num_simulations)
    
    # Calculate total goals for each simulation
    total_goals = home_scores + away_scores
    
    # Count outcomes
    home_wins = np.sum(home_scores > away_scores)
    draws = np.sum(home_scores == away_scores)
    away_wins = np.sum(home_scores < away_scores)
    
    # Count over/under 2.5 goals
    over_2_5 = np.sum(total_goals > 2.5)
    under_2_5 = np.sum(total_goals <= 2.5)
    
    # Convert to probabilities
    home_win_prob = home_wins / num_simulations
    draw_prob = draws / num_simulations
    away_win_prob = away_wins / num_simulations
    over_2_5_prob = over_2_5 / num_simulations
    under_2_5_prob = under_2_5 / num_simulations
    
    return {
        'home_win': home_win_prob,
        'draw': draw_prob,
        'away_win': away_win_prob,
        'over_2_5': over_2_5_prob,
        'under_2_5': under_2_5_prob
    }

# Add these probabilities to your prediction dataframe
backtest_df['probs'] = backtest_df.apply(
    lambda row: simulate_match_outcomes(row['predicted_home_goals'], row['predicted_away_goals']), 
    axis=1
)

# Extract individual probabilities to separate columns
backtest_df['home_win_prob'] = backtest_df['probs'].apply(lambda x: x['home_win'])
backtest_df['draw_prob'] = backtest_df['probs'].apply(lambda x: x['draw'])
backtest_df['away_win_prob'] = backtest_df['probs'].apply(lambda x: x['away_win'])
backtest_df['over_2_5_prob'] = backtest_df['probs'].apply(lambda x: x['over_2_5'])
backtest_df['under_2_5_prob'] = backtest_df['probs'].apply(lambda x: x['under_2_5'])

# Drop the intermediate dictionary column
backtest_df = backtest_df.drop('probs', axis=1)

append_df_to_sqlite_table(backtest_df, 'match_db.db', 'xgb_backtest_0065_001_prem20242025', key_columns=['match_id'])

backtest_df

No new records to append.


,match_id,predicted_home_goals,actual_home_goals,predicted_away_goals,actual_away_goals,bet365_home_odds,bet365_draw_odds,bet365_away_odds,bet365_u25_odds,bet365_o25_odds,home_win_prob,draw_prob,away_win_prob,over_2_5_prob,under_2_5_prob
0,Arsenal - Aston Villa_20250118,1.590694,2,1.074542,2,1.50,4.10,7.00,2.03,1.87,0.4955,0.2424,0.2621,0.5042,0.4958
1,Arsenal - Brighton_20240831,1.843244,1,1.144221,1,1.33,5.50,8.50,2.50,1.53,0.5316,0.2305,0.2379,0.5777,0.4223
2,Arsenal - Chelsea_20250316,1.742448,1,1.133815,0,1.80,3.60,4.50,1.92,1.86,0.5140,0.2373,0.2487,0.5455,0.4545
3,Arsenal - Everton_20241214,2.030706,0,0.932608,0,1.22,6.50,12.00,2.20,1.67,0.6284,0.2100,0.1616,0.5670,0.4330
4,Arsenal - Ipswich_20241227,2.246113,1,0.899718,0,1.14,9.00,15.00,3.00,1.40,0.6777,0.1839,0.1384,0.6046,0.3954
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,Wolves - Man City_20241020,1.058062,1,2.087760,2,8.00,6.00,1.33,2.75,1.44,0.1818,0.2056,0.6126,0.6113,0.3887
285,Wolves - Man United_20241226,1.316546,2,1.455159,0,3.90,3.70,1.91,2.20,1.67,0.3405,0.2564,0.4031,0.5217,0.4783
286,Wolves - Newcastle_20240915,1.290198,1,1.855103,2,3.40,3.50,2.10,2.38,1.57,0.2699,0.2278,0.5023,0.6021,0.3979
287,Wolves - Nott'm Forest_20250106,1.245404,0,1.231543,3,3.30,3.20,2.30,1.73,2.10,0.3702,0.2711,0.3587,0.4505,0.5495


In [81]:
def kelly_expected(probability, decimal_odds, fraction = 1.0):
    if decimal_odds <= 1 or probability <= 0 or probability >= 1:
        return 0, 0
        
    # Calculate Kelly Criterion
    q = 1 - probability  # probability of losing
    kelly = (probability * (decimal_odds - 1) - q) / (decimal_odds - 1) # win rate * potential profit subtracitng prob of losing. Divided by potentail profit
    kelly = kelly * fraction  # Apply fractional Kelly
    kelly = max(0, kelly)  # No negative bets
    
    # Calculate Expected Value
    ev = (probability * (decimal_odds - 1)) - (1 - probability)
    ev = ev  # Convert to percentage
    
    return kelly, ev

prob = 0.6398
odds = 1.67
fraction = 0.25

bet_size, ev = kelly_expected(prob, odds, fraction)
print(f"Optimal bet size: {bet_size:.1%}")
print(f"Expected Value: {ev:.1%}")


Optimal bet size: 2.6%
Expected Value: 6.8%


In [82]:
home_win_df = pd.read_sql_query("""SELECT match_id, actual_home_goals, actual_away_goals, home_win_prob, bet365_home_odds FROM xgb_backtest_0065_001_prem20242025""", conn)

home_win_df["home_win?"] = (home_win_df["actual_home_goals"] > home_win_df["actual_away_goals"]).astype(int)

# Calculate implied probability from Bet365 odds
home_win_df['implied_prob'] = (1 / home_win_df['bet365_home_odds'])

results = home_win_df.apply(
    lambda row: kelly_expected(row['home_win_prob'], row['bet365_home_odds'], fraction=0.25),
    axis=1
)

# Extract Kelly and EV values into separate columns
home_win_df['kelly_stake'] = results.apply(lambda x: x[0])
home_win_df['ev_percent'] = results.apply(lambda x: x[1])
home_win_df['is_value_bet'] = (home_win_df['ev_percent'] > 0.00) 
#print(home_win_df)

home_win_df['profit'] = home_win_df.apply(
    lambda row: row['kelly_stake'] * (row['bet365_home_odds'] - 1) if row['is_value_bet'] and row['home_win?'] == 1 
              else (-row['kelly_stake'] if row['is_value_bet'] else 0),  # Lose Kelly stake for lost bets
    axis=1
)

# Display summary statistics
value_bets = home_win_df[home_win_df['is_value_bet']]
print(f"Total matches analyzed: {len(home_win_df)}")
print(f"Value bets found: {len(value_bets)} ({len(value_bets)/len(home_win_df):.1%} of total)")

# Add average calculations for all bets
avg_ev_all = home_win_df['ev_percent'].mean()
avg_odds_all = home_win_df['bet365_home_odds'].mean()
avg_implied_prob_all = home_win_df['implied_prob'].mean()
avg_model_prob_all = home_win_df['home_win_prob'].mean()

print(f"Average EV for all bets: {avg_ev_all:.2%}")
print(f"Average Bet365 odds for all bets: {avg_odds_all:.2f}")
print(f"Average implied probability for all bets: {avg_implied_prob_all:.2%}")
print(f"Average model probability for all bets: {avg_model_prob_all:.2%}")

if len(value_bets) > 0:
    # Calculate performance of value bets
    won_bets = value_bets['home_win?'].sum()
    profit = value_bets['profit'].sum()
    avg_ev_value = value_bets['ev_percent'].mean()
    avg_odds_value = value_bets['bet365_home_odds'].mean()
    avg_implied_prob_value = value_bets['implied_prob'].mean()
    avg_model_prob_value = value_bets['home_win_prob'].mean()
    
    print(f"\nValue bets statistics:")
    print(f"Value bets won: {won_bets}/{len(value_bets)} ({won_bets/len(value_bets):.1%})")
    print(f"Total profit: {profit:.2f} units")
    print(f"ROI: {profit/len(value_bets)*100:.2f}%")
    print(f"Average EV for value bets: {avg_ev_value:.2%}")
    print(f"Average Bet365 odds for value bets: {avg_odds_value:.2f}")
    print(f"Average implied probability for value bets: {avg_implied_prob_value:.2%}")
    print(f"Average model probability for value bets: {avg_model_prob_value:.2%}")
    print(f"Average edge (model prob - implied prob): {(avg_model_prob_value - avg_implied_prob_value):.2%}")

Total matches analyzed: 289
Value bets found: 85 (29.4% of total)
Average EV for all bets: -6.01%
Average Bet365 odds for all bets: 2.77
Average implied probability for all bets: 46.59%
Average model probability for all bets: 40.23%

Value bets statistics:
Value bets won: 15/85 (17.6%)
Total profit: -0.80 units
ROI: -0.94%
Average EV for value bets: 23.71%
Average Bet365 odds for value bets: 4.50
Average implied probability for value bets: 26.41%
Average model probability for value bets: 31.03%
Average edge (model prob - implied prob): 4.62%


In [83]:
away_win_df = pd.read_sql_query("""SELECT match_id, actual_home_goals, actual_away_goals, away_win_prob, bet365_away_odds FROM xgb_backtest_0065_001_prem20242025""", conn)

away_win_df["away_win?"] = (away_win_df["actual_away_goals"] > away_win_df["actual_home_goals"]).astype(int)

results = away_win_df.apply(
    lambda row: kelly_expected(row['away_win_prob'], row['bet365_away_odds'], fraction=0.25),
    axis=1
)

# Extract Kelly and EV values into separate columns
away_win_df['kelly_stake'] = results.apply(lambda x: x[0])
away_win_df['ev_percent'] = results.apply(lambda x: x[1])
away_win_df['is_value_bet'] = (away_win_df['ev_percent'] > 0.00)

away_win_df['profit'] = away_win_df.apply(
    lambda row: row['kelly_stake'] * (row['bet365_away_odds'] - 1) if row['is_value_bet'] and row['away_win?'] == 1 
              else (-row['kelly_stake'] if row['is_value_bet'] else 0),  # Lose Kelly stake for lost bets
    axis=1
)

# Display summary statistics
value_bets = away_win_df[away_win_df['is_value_bet']]
print(f"Total matches analyzed: {len(away_win_df)}")
print(f"Value bets found: {len(value_bets)} ({len(value_bets)/len(away_win_df):.1%} of total)")

if len(value_bets) > 0:
    # Calculate performance of value bets
    won_bets = value_bets['away_win?'].sum()
    profit = value_bets['profit'].sum()
    
    print(f"Value bets won: {won_bets}/{len(value_bets)} ({won_bets/len(value_bets):.1%})")
    print(f"Total profit: {profit:.2f} units")
    print(f"ROI: {profit/len(value_bets)*100:.2f}%")

Total matches analyzed: 289
Value bets found: 193 (66.8% of total)
Value bets won: 53/193 (27.5%)
Total profit: 0.45 units
ROI: 0.23%


In [84]:
draw_df = pd.read_sql_query("""SELECT match_id, actual_home_goals, actual_away_goals, draw_prob, bet365_draw_odds FROM xgb_backtest_0065_001_prem20242025""", conn)

draw_df["draw?"] = (draw_df["actual_away_goals"] == draw_df["actual_home_goals"]).astype(int)

results = draw_df.apply(
    lambda row: kelly_expected(row['draw_prob'], row['bet365_draw_odds'], fraction=0.25),
    axis=1
)

# Extract Kelly and EV values into separate columns
draw_df['kelly_stake'] = results.apply(lambda x: x[0])
draw_df['ev_percent'] = results.apply(lambda x: x[1])
draw_df['is_value_bet'] = draw_df['ev_percent'] > 0.02

draw_df['profit'] = draw_df.apply(
    lambda row: row['kelly_stake'] * (row['bet365_draw_odds'] - 1) if row['is_value_bet'] and row['draw?'] == 1 
              else (-row['kelly_stake'] if row['is_value_bet'] else 0),  # Lose Kelly stake for lost bets
    axis=1
)

# Display summary statistics
value_bets = draw_df[draw_df['is_value_bet']]
print(f"Total matches analyzed: {len(draw_df)}")
print(f"Value bets found: {len(value_bets)} ({len(value_bets)/len(draw_df):.1%} of total)")

if len(value_bets) > 0:
    # Calculate performance of value bets
    won_bets = value_bets['draw?'].sum()
    profit = value_bets['profit'].sum()
    
    print(f"Value bets won: {won_bets}/{len(value_bets)} ({won_bets/len(value_bets):.1%})")
    print(f"Total profit: {profit:.2f} units")
    print(f"ROI: {profit/len(value_bets)*100:.2f}%")

Total matches analyzed: 289
Value bets found: 80 (27.7% of total)
Value bets won: 18/80 (22.5%)
Total profit: -0.07 units
ROI: -0.09%


In [85]:
over_25_df = pd.read_sql_query("""SELECT match_id, actual_home_goals, actual_away_goals, over_2_5_prob, bet365_o25_odds FROM xgb_backtest_0065_001_prem20242025""", conn)

over_25_df["over_25?"] = (over_25_df["actual_away_goals"] + over_25_df["actual_home_goals"] > 2.5).astype(int)

results = over_25_df.apply(
    lambda row: kelly_expected(row['over_2_5_prob'], row['bet365_o25_odds'], fraction=0.25),
    axis=1
)

# Extract Kelly and EV values into separate columns
over_25_df['kelly_stake'] = results.apply(lambda x: x[0])
over_25_df['ev_percent'] = results.apply(lambda x: x[1])
over_25_df['is_value_bet'] = (over_25_df['ev_percent'] > 0.02) 

over_25_df['profit'] = over_25_df.apply(
    lambda row: row['kelly_stake'] * (row['bet365_o25_odds'] - 1) if row['is_value_bet'] and row['over_25?'] == 1 
              else (-row['kelly_stake'] if row['is_value_bet'] else 0),  # Lose Kelly stake for lost bets
    axis=1
)

# Display summary statistics
value_bets = over_25_df[over_25_df['is_value_bet']]
print(f"Total matches analyzed: {len(over_25_df)}")
print(f"Value bets found: {len(value_bets)} ({len(value_bets)/len(over_25_df):.1%} of total)")

if len(value_bets) > 0:
    # Calculate performance of value bets
    won_bets = value_bets['over_25?'].sum()
    profit = value_bets['profit'].sum()
    
    print(f"Value bets won: {won_bets}/{len(value_bets)} ({won_bets/len(value_bets):.1%})")
    print(f"Total profit: {profit:.2f} units")
    print(f"ROI: {profit/len(value_bets)*100:.2f}%")

Total matches analyzed: 289
Value bets found: 33 (11.4% of total)
Value bets won: 16/33 (48.5%)
Total profit: -0.07 units
ROI: -0.22%


In [86]:
under_25_df = pd.read_sql_query("""SELECT match_id, actual_home_goals, actual_away_goals, under_2_5_prob, bet365_u25_odds FROM xgb_backtest_0065_001_prem20242025""", conn)

under_25_df["under_25?"] = (under_25_df["actual_away_goals"] + under_25_df["actual_home_goals"] <= 2.5).astype(int)

results = under_25_df.apply(
    lambda row: kelly_expected(row['under_2_5_prob'], row['bet365_u25_odds'], fraction=0.25),
    axis=1
)

# Extract Kelly and EV values into separate columns
under_25_df['kelly_stake'] = results.apply(lambda x: x[0])
under_25_df['ev_percent'] = results.apply(lambda x: x[1])
under_25_df['is_value_bet'] = under_25_df['ev_percent'] > 0.02

under_25_df['profit'] = under_25_df.apply(
    lambda row: row['kelly_stake'] * (row['bet365_u25_odds'] - 1) if row['is_value_bet'] and row['under_25?'] == 1 
              else (-row['kelly_stake'] if row['is_value_bet'] else 0),  # Lose Kelly stake for lost bets
    axis=1
)

# Display summary statistics
value_bets = under_25_df[under_25_df['is_value_bet']]
print(f"Total matches analyzed: {len(under_25_df)}")
print(f"Value bets found: {len(value_bets)} ({len(value_bets)/len(under_25_df):.1%} of total)")

if len(value_bets) > 0:
    # Calculate performance of value bets
    won_bets = value_bets['under_25?'].sum()
    profit = value_bets['profit'].sum()
    
    print(f"Value bets won: {won_bets}/{len(value_bets)} ({won_bets/len(value_bets):.1%})")
    print(f"Total profit: {profit:.2f} units")
    print(f"ROI: {profit/len(value_bets)*100:.2f}%")

Total matches analyzed: 289
Value bets found: 130 (45.0% of total)
Value bets won: 42/130 (32.3%)
Total profit: -0.11 units
ROI: -0.08%


# Asian Handicap Testing

In [87]:
ah_backtest_df = pd.read_sql_query("""
    SELECT
        preds.match_id,
        preds.predicted_home_goals,
        preds.actual_home_goals,
        preds.predicted_away_goals,
        preds.actual_away_goals,
        eo.AHh,
        eo.B365AHH,
        eo.B365AHA,
        eo.AHCh,
        eo.B365CAHH,
        eo.B365CAHA
    FROM
        xgb_pred_0065_001_prem20242025 preds
    JOIN
        epl_odds eo
            ON eo.match_id = preds.match_id
                                
    """, conn)

ah_backtest_df

,match_id,predicted_home_goals,actual_home_goals,predicted_away_goals,actual_away_goals,AHh,B365AHH,B365AHA,AHCh,B365CAHH,B365CAHA
0,Arsenal - Aston Villa_20250118,1.590694,2,1.074542,2,-1.00,1.87,2.03,-1.00,1.98,1.92
1,Arsenal - Brighton_20240831,1.843244,1,1.144221,1,-1.50,2.01,1.89,-1.50,2.08,1.85
2,Arsenal - Chelsea_20250316,1.742448,1,1.133815,0,-0.75,2.06,1.84,-0.75,1.95,1.95
3,Arsenal - Everton_20241214,2.030706,0,0.932608,0,-1.75,1.95,1.95,-1.75,2.05,1.88
4,Arsenal - Ipswich_20241227,2.246113,1,0.899718,0,-2.25,2.04,1.89,-2.25,2.02,1.91
...,...,...,...,...,...,...,...,...,...,...,...
284,Wolves - Man City_20241020,1.058062,1,2.087760,2,1.50,2.02,1.88,1.50,1.85,2.08
285,Wolves - Man United_20241226,1.316546,2,1.455159,0,0.50,1.95,1.95,0.75,1.87,2.06
286,Wolves - Newcastle_20240915,1.290198,1,1.855103,2,0.25,2.06,1.84,0.50,1.85,2.08
287,Wolves - Nott'm Forest_20250106,1.245404,0,1.231543,3,0.25,1.91,1.99,0.50,1.88,2.02


In [88]:
def calculate_ah_probs(df, random_seed=26, n_sims=50000):
    # Create a copy to avoid modifying the original
    result_df = df.copy()
    
    # Define a function to apply to each row
    def calc_probs_for_row(row):
        np.random.seed(random_seed)
        
        home_pred_goals = row['predicted_home_goals']
        away_pred_goals = row['predicted_away_goals']
        line = row['AHh']
        
        home_goals = np.random.poisson(home_pred_goals, size=n_sims)
        away_goals = np.random.poisson(away_pred_goals, size=n_sims)
        home_goal_diff = home_goals - away_goals
        
        if line % 0.5 == 0.25 or line % 0.5 == -0.25:
            # Split into two lines
            lower_line = np.floor(line)
            upper_line = np.floor(line) + 0.5
            
            # If it's a negative quarter (e.g., -0.25), swap the order
            if line % 1 == -0.25:
                lower_line = np.ceil(line)
                upper_line = np.ceil(line) - 0.5
            
            # Calculate probabilities for each part
            # For lower line
            lower_wins = np.sum((home_goal_diff + lower_line) > 0)
            lower_draws = np.sum((home_goal_diff + lower_line) == 0)
            lower_losses = np.sum((home_goal_diff + lower_line) < 0)
            
            # For upper line
            upper_wins = np.sum((home_goal_diff + upper_line) > 0)
            upper_draws = np.sum((home_goal_diff + upper_line) == 0)
            upper_losses = np.sum((home_goal_diff + upper_line) < 0)
            
            # Combine results (50% weight to each line)
            home_prob = ((lower_wins + 0.5 * lower_draws) + (upper_wins + 0.5 * upper_draws)) / (2 * n_sims)
            away_prob = ((lower_losses + 0.5 * lower_draws) + (upper_losses + 0.5 * upper_draws)) / (2 * n_sims)
        
        else:
            # Handle regular lines (0, whole, half)
            home_wins = np.sum((home_goal_diff + line) > 0)
            draws = np.sum((home_goal_diff + line) == 0)
            away_wins = np.sum((home_goal_diff + line) < 0)
            
            home_prob = (home_wins + 0.5 * draws) / n_sims
            away_prob = (away_wins + 0.5 * draws) / n_sims
        
        return pd.Series({'home_win_prob': home_prob, 'away_win_prob': away_prob})
    
    # Apply the function to each row
    probs = result_df.apply(calc_probs_for_row, axis=1)
    
    # Add the calculated probabilities to the result dataframe
    result_df['home_win_prob'] = probs['home_win_prob']
    result_df['away_win_prob'] = probs['away_win_prob']
    
    return result_df

ah_backtest_df = calculate_ah_probs(ah_backtest_df)

ah_backtest_df

,match_id,predicted_home_goals,actual_home_goals,predicted_away_goals,actual_away_goals,AHh,B365AHH,B365AHA,AHCh,B365CAHH,B365CAHA,home_win_prob,away_win_prob
0,Arsenal - Aston Villa_20250118,1.590694,2,1.074542,2,-1.00,1.87,2.03,-1.00,1.98,1.92,0.377230,0.622770
1,Arsenal - Brighton_20240831,1.843244,1,1.144221,1,-1.50,2.01,1.89,-1.50,2.08,1.85,0.305520,0.694480
2,Arsenal - Chelsea_20250316,1.742448,1,1.133815,0,-0.75,2.06,1.84,-0.75,1.95,1.95,0.460385,0.539615
3,Arsenal - Everton_20241214,2.030706,0,0.932608,0,-1.75,1.95,1.95,-1.75,2.05,1.88,0.339585,0.660415
4,Arsenal - Ipswich_20241227,2.246113,1,0.899718,0,-2.25,2.04,1.89,-2.25,2.02,1.91,0.210130,0.789870
...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,Wolves - Man City_20241020,1.058062,1,2.087760,2,1.50,2.02,1.88,1.50,1.85,2.08,0.625780,0.374220
285,Wolves - Man United_20241226,1.316546,2,1.455159,0,0.50,1.95,1.95,0.75,1.87,2.06,0.599540,0.400460
286,Wolves - Newcastle_20240915,1.290198,1,1.855103,2,0.25,2.06,1.84,0.50,1.85,2.08,0.440675,0.559325
287,Wolves - Nott'm Forest_20250106,1.245404,0,1.231543,3,0.25,1.91,1.99,0.50,1.88,2.02,0.573710,0.426290


In [89]:
def calculate_ah_results(df):
    # Create a copy to avoid modifying the original
    result_df = df.copy()
    
    # Define a function to calculate the result for a single row
    def calc_result_for_row(row):
        home_goals = row['actual_home_goals']
        away_goals = row['actual_away_goals']
        line = row['AHh']
        
        # Calculate adjusted home score
        adjusted_home = home_goals + line
        goal_diff = adjusted_home - away_goals
        
        # Result values for home bet
        home_result = 0
        
        # For quarter lines
        if line % 0.5 == 0.25 or line % 0.5 == -0.25:
            lower_line = np.floor(line)
            upper_line = np.floor(line) + 0.5
            
            # If it's a negative quarter (e.g., -0.25), swap the order
            if line % 1 == -0.25:
                lower_line = np.ceil(line)
                upper_line = np.ceil(line) - 0.5
                
            lower_diff = (home_goals + lower_line) - away_goals
            upper_diff = (home_goals + upper_line) - away_goals
            
            # Calculate combined result
            if lower_diff > 0 and upper_diff > 0:
                home_result = 1.0  # Full win
            elif lower_diff == 0 and upper_diff > 0:
                home_result = 0.75  # Half-win, half-push
            elif lower_diff > 0 and upper_diff == 0:
                home_result = 0.75  # Half-win, half-push
            elif lower_diff == 0 and upper_diff == 0:
                home_result = 0.5  # Push
            elif lower_diff < 0 and upper_diff == 0:
                home_result = 0.25  # Half-push, half-loss
            elif lower_diff == 0 and upper_diff < 0:
                home_result = 0.25  # Half-push, half-loss
            else:
                home_result = 0.0  # Full loss
        
        # For whole or half lines
        else:
            if goal_diff > 0:
                home_result = 1.0  # Win
            elif goal_diff == 0:
                home_result = 0.5  # Push
            else:
                home_result = 0.0  # Loss
        
        # Away result is the complement of home result
        away_result = 1.0 - home_result
        
        return pd.Series({'home_ah_result': home_result, 'away_ah_result': away_result})
    
    # Apply the function to each row
    results = result_df.apply(calc_result_for_row, axis=1)
    
    # Add the calculated results to the result dataframe
    result_df['home_ah_result'] = results['home_ah_result']
    result_df['away_ah_result'] = results['away_ah_result']
    
    return result_df

ah_backtest_df = calculate_ah_results(ah_backtest_df)

ah_backtest_df

,match_id,predicted_home_goals,actual_home_goals,predicted_away_goals,actual_away_goals,AHh,B365AHH,B365AHA,AHCh,B365CAHH,B365CAHA,home_win_prob,away_win_prob,home_ah_result,away_ah_result
0,Arsenal - Aston Villa_20250118,1.590694,2,1.074542,2,-1.00,1.87,2.03,-1.00,1.98,1.92,0.377230,0.622770,0.00,1.00
1,Arsenal - Brighton_20240831,1.843244,1,1.144221,1,-1.50,2.01,1.89,-1.50,2.08,1.85,0.305520,0.694480,0.00,1.00
2,Arsenal - Chelsea_20250316,1.742448,1,1.133815,0,-0.75,2.06,1.84,-0.75,1.95,1.95,0.460385,0.539615,0.75,0.25
3,Arsenal - Everton_20241214,2.030706,0,0.932608,0,-1.75,1.95,1.95,-1.75,2.05,1.88,0.339585,0.660415,0.00,1.00
4,Arsenal - Ipswich_20241227,2.246113,1,0.899718,0,-2.25,2.04,1.89,-2.25,2.02,1.91,0.210130,0.789870,0.00,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,Wolves - Man City_20241020,1.058062,1,2.087760,2,1.50,2.02,1.88,1.50,1.85,2.08,0.625780,0.374220,1.00,0.00
285,Wolves - Man United_20241226,1.316546,2,1.455159,0,0.50,1.95,1.95,0.75,1.87,2.06,0.599540,0.400460,1.00,0.00
286,Wolves - Newcastle_20240915,1.290198,1,1.855103,2,0.25,2.06,1.84,0.50,1.85,2.08,0.440675,0.559325,0.00,1.00
287,Wolves - Nott'm Forest_20250106,1.245404,0,1.231543,3,0.25,1.91,1.99,0.50,1.88,2.02,0.573710,0.426290,0.00,1.00


In [90]:
# Calculate Kelly and EV for home bets
home_results = ah_backtest_df.apply(
    lambda row: kelly_expected(row['home_win_prob'], row['B365AHH'], fraction=0.25),
    axis=1
)

# Calculate Kelly and EV for away bets
away_results = ah_backtest_df.apply(
    lambda row: kelly_expected(row['away_win_prob'], row['B365AHA'], fraction=0.25),
    axis=1
)

# Extract values into columns
ah_backtest_df['home_kelly_stake'] = home_results.apply(lambda x: x[0])
ah_backtest_df['home_ev_percent'] = home_results.apply(lambda x: x[1])
ah_backtest_df['away_kelly_stake'] = away_results.apply(lambda x: x[0])
ah_backtest_df['away_ev_percent'] = away_results.apply(lambda x: x[1])

# Determine which side has better EV
ah_backtest_df['better_side'] = ah_backtest_df.apply(
    lambda row: 'home' if row['home_ev_percent'] > row['away_ev_percent'] else 'away',
    axis=1
)


ah_backtest_df

,match_id,predicted_home_goals,actual_home_goals,predicted_away_goals,actual_away_goals,AHh,B365AHH,B365AHA,AHCh,B365CAHH,B365CAHA,home_win_prob,away_win_prob,home_ah_result,away_ah_result,home_kelly_stake,home_ev_percent,away_kelly_stake,away_ev_percent,better_side
0,Arsenal - Aston Villa_20250118,1.590694,2,1.074542,2,-1.00,1.87,2.03,-1.00,1.98,1.92,0.377230,0.622770,0.00,1.00,0.000000,-0.294580,0.064132,0.264223,away
1,Arsenal - Brighton_20240831,1.843244,1,1.144221,1,-1.50,2.01,1.89,-1.50,2.08,1.85,0.305520,0.694480,0.00,1.00,0.000000,-0.385905,0.087800,0.312567,away
2,Arsenal - Chelsea_20250316,1.742448,1,1.133815,0,-0.75,2.06,1.84,-0.75,1.95,1.95,0.460385,0.539615,0.75,0.25,0.000000,-0.051607,0.000000,-0.007108,away
3,Arsenal - Everton_20241214,2.030706,0,0.932608,0,-1.75,1.95,1.95,-1.75,2.05,1.88,0.339585,0.660415,0.00,1.00,0.000000,-0.337809,0.075739,0.287809,away
4,Arsenal - Ipswich_20241227,2.246113,1,0.899718,0,-2.25,2.04,1.89,-2.25,2.02,1.91,0.210130,0.789870,0.00,1.00,0.000000,-0.571335,0.138442,0.492854,away
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,Wolves - Man City_20241020,1.058062,1,2.087760,2,1.50,2.02,1.88,1.50,1.85,2.08,0.625780,0.374220,1.00,0.00,0.064724,0.264076,0.000000,-0.296466,home
285,Wolves - Man United_20241226,1.316546,2,1.455159,0,0.50,1.95,1.95,0.75,1.87,2.06,0.599540,0.400460,1.00,0.00,0.044501,0.169103,0.000000,-0.219103,home
286,Wolves - Newcastle_20240915,1.290198,1,1.855103,2,0.25,2.06,1.84,0.50,1.85,2.08,0.440675,0.559325,0.00,1.00,0.000000,-0.092210,0.008678,0.029158,away
287,Wolves - Nott'm Forest_20250106,1.245404,0,1.231543,3,0.25,1.91,1.99,0.50,1.88,2.02,0.573710,0.426290,0.00,1.00,0.026315,0.095786,0.000000,-0.151683,home


In [91]:
# Helper function to calculate profit based on chosen side
def calculate_bet_profit(row):
    if not row['is_value_bet']:
        return 0
        
    if row['better_side'] == 'home':
        stake = row['home_kelly_stake']
        odds = row['B365AHH']
        result = row['home_ah_result']
    else:  # away
        stake = row['away_kelly_stake']
        odds = row['B365AHA']
        result = row['away_ah_result']
    
    if result == 1.0:  # Full win
        return stake * (odds - 1)
    elif result == 0.75:  # 3/4 win
        return stake * (odds - 1) * 0.75
    elif result == 0.5:  # Push
        return 0
    elif result == 0.25:  # 1/4 win
        return stake * (odds - 1) * 0.25
    else:  # Full loss
        return -stake

# Determine if it's a value bet based on the better side
ah_backtest_df['is_value_bet'] = ah_backtest_df.apply(
    lambda row: row['home_ev_percent'] > 0.02 if row['better_side'] == 'home' 
                else row['away_ev_percent'] > 0.02,
    axis=1
)

# Calculate profit based on which side we bet on
ah_backtest_df['profit'] = ah_backtest_df.apply(
    lambda row: calculate_bet_profit(row),
    axis=1
)

# Display summary statistics
value_bets = ah_backtest_df[ah_backtest_df['is_value_bet']]
print(f"Total matches analyzed: {len(ah_backtest_df)}")
print(f"Value bets found: {len(value_bets)} ({len(value_bets)/len(ah_backtest_df):.1%} of total)")
print(f"Home bets: {sum(value_bets['better_side'] == 'home')}")
print(f"Away bets: {sum(value_bets['better_side'] == 'away')}")

if len(value_bets) > 0:
    # Group results by the chosen side
    home_bets = value_bets[value_bets['better_side'] == 'home']
    away_bets = value_bets[value_bets['better_side'] == 'away']
    
    # Overall performance
    profit = value_bets['profit'].sum()
    pushes = sum((value_bets['better_side'] == 'home') & (value_bets['home_ah_result'] == 0.5)) + \
             sum((value_bets['better_side'] == 'away') & (value_bets['away_ah_result'] == 0.5))
    
    # For ROI, we exclude pushes
    bets_with_outcome = len(value_bets) - pushes
    
    print(f"\nOverall performance:")
    print(f"Total profit: {profit:.2f} units")
    if bets_with_outcome > 0:
        print(f"ROI: {profit/bets_with_outcome*100:.2f}% (excluding pushes)")
    
    # Performance by side
    if len(home_bets) > 0:
        home_profit = home_bets['profit'].sum()
        home_pushes = sum(home_bets['home_ah_result'] == 0.5)
        home_bets_with_outcome = len(home_bets) - home_pushes
        
        print(f"\nHome bets performance:")
        print(f"Profit: {home_profit:.2f} units")
        if home_bets_with_outcome > 0:
            print(f"ROI: {home_profit/home_bets_with_outcome*100:.2f}%")
    
    if len(away_bets) > 0:
        away_profit = away_bets['profit'].sum()
        away_pushes = sum(away_bets['away_ah_result'] == 0.5)
        away_bets_with_outcome = len(away_bets) - away_pushes
        
        print(f"\nAway bets performance:")
        print(f"Profit: {away_profit:.2f} units")
        if away_bets_with_outcome > 0:
            print(f"ROI: {away_profit/away_bets_with_outcome*100:.2f}%")

Total matches analyzed: 289
Value bets found: 252 (87.2% of total)
Home bets: 54
Away bets: 198

Overall performance:
Total profit: 3.41 units
ROI: 1.44% (excluding pushes)

Home bets performance:
Profit: -0.35 units
ROI: -0.68%

Away bets performance:
Profit: 3.76 units
ROI: 2.02%
